<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #9: Modelado de temas

Guillermo Andres Cardona
cod:616202023

In [1]:
import re
import pandas as pd 

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim_models
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\Memo\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Memo\anaconda3\lib\site-packages\gensim\corpora\dictionary.py:12: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Mapping, defaultdict
C:\Users\Memo\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [3]:
# Cargar datos
def cargarDatos(archivo):
    data = pd.read_csv(archivo)
    print(data.sample(10))
    return data;

def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    #texto = texto.split() # Tokenizar
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    #texto = " ".join(texto)
    return texto

In [4]:
datosg = cargarDatos("C:/Users/Memo/Desktop/npl/taller_9/reviews_vidjew_es.csv")

      review_id          product_id          reviewer_id  stars  \
479  es_0114406  product_es_0889345  reviewer_es_0510860      2   
732  es_0982634  product_es_0945888  reviewer_es_0100294      2   
589  es_0048586  product_es_0303197  reviewer_es_0806281      3   
917  es_0496055  product_es_0949534  reviewer_es_0034912      4   
544  es_0796327  product_es_0741008  reviewer_es_0123114      1   
215  es_0938485  product_es_0350759  reviewer_es_0241278      3   
365  es_0146500  product_es_0300747  reviewer_es_0704796      2   
885  es_0432159  product_es_0050166  reviewer_es_0038001      4   
63   es_0032942  product_es_0432847  reviewer_es_0021862      4   
55   es_0627834  product_es_0612149  reviewer_es_0061610      5   

                                           review_body  \
479  El peor juego que he visto, no tienes nada que...   
732  Color y dibujo bien, pero cuesta mucho de cuad...   
589  Lo veo algo caro para la calidad que tiene. Ve...   
917  Estoy disfrutando a tope 

###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [5]:
datosg['Pre-Processed'] = datosg['review_body'].apply(lambda datosg: pre_procesado(datosg))
datosg.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,Pre-Processed
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


In [11]:
# Crear una representación de los documentos en forma de diccionario
dictionary = Dictionary(datosg['Pre-Processed'].values)

# Filtrar palabras muy frecuentes o infrecuentes
dictionary.filter_extremes(no_below=7, no_above=0.4) 

# Corpus
corpus = [dictionary.doc2bow(text) for text in datosg['Pre-Processed'].values]


###  `[10 pts]` Punto 2: Modelo de LDA

In [12]:
# Entrenar el modelo
model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = 4, passes=35)
model.print_topics(num_words=8)

[(0,
  '0.062*"bien" + 0.028*"juego" + 0.024*"parece" + 0.019*"calidad" + 0.018*"caja" + 0.018*"precio" + 0.018*"dos" + 0.017*"cadena"'),
 (1,
  '0.039*"calidad" + 0.028*"bien" + 0.024*"pulsera" + 0.018*"foto" + 0.016*"perfecto" + 0.015*"bueno" + 0.014*"cumple" + 0.014*"producto"'),
 (2,
  '0.050*"si" + 0.030*"bastante" + 0.021*"juego" + 0.020*"regalo" + 0.016*"ps" + 0.016*"gustado" + 0.016*"tal" + 0.014*"mando"'),
 (3,
  '0.031*"llegado" + 0.030*"producto" + 0.030*"día" + 0.029*"precio" + 0.027*"calidad" + 0.021*"buena" + 0.019*"juego" + 0.017*"buen"')]

###  `[25 pts]` Punto 3: Visualización de LDA

In [13]:
lda_display = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
pyLDAvis.display(lda_display)

###  `[3 pts]` Preguntas

- ¿Qué conclusiones puede sacar de los resultados de los últimos tres talleres, usando diferentes modelos pero los mismos datos?

se ha trabajado con la misma data en los 3 talleres, mi percepecion es que en definitiva los modelos supervisados bien estructurados son mas acertados y los resultados son mas eficientes, para este modelo lda, teniendo en cuenta que las data me muestra dos categorias su clasificacion es medianamente acertada, su visualizacion es muy dinamica para la toma de decisiones.